Compiling code

1.   Object detection in Video
2.   Eye blink detection
3.   Face recognition



Mounting google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
path = '/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled'
os.chdir(path)

Mounted at /content/drive


Importing the required libraries

In [4]:
!pip install ultralytics
!pip install deepface
# !pip install "/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled/Dependencies/dlib-19.24.1-cp311-cp311-win_amd64.whl"

In [5]:
import numpy as np
import pandas as pd

import cv2
from google.colab.patches import cv2_imshow
from ultralytics import YOLO

import dlib
from imutils import face_utils
from scipy.spatial import distance as dist

from deepface import DeepFace

import time

# Start timer
tic = time.time()

# Load the pre-trained model for YOLOv8
model = YOLO('yolov8m.pt')

# Path to the video
video_path = '/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled/inputs/Ashish.mp4'

# Run the video through the model
results = model(source=video_path, show=True, conf=0.4, save=True)

tc = time.time()
print(tc-tic)

# Initialize an empty set to store unique object names
detected_objects = set()

# Process the results
for result in results:
    for detection in result.boxes.data:
        class_id = int(detection[5])
        class_name = model.names[class_id]
        detected_objects.add(class_name)

# Convert the set to a list to get the final list of detected objects
objects_list = list(detected_objects)
print(objects_list)
print('\n')

# Check for unauthorized objects
objects_of_interest = ["cell phone"]  # "laptop"
if any(obj in objects_list for obj in objects_of_interest):
    print("UNAUTHORISED ACCESS: Phone/Laptop is detected.")
else:
    print('Moving to eye-blink detection.\n')

    def eye_aspect_ratio(eye):
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])
        C = dist.euclidean(eye[0], eye[3])
        ear = (A + B) / (2.0 * C)
        return ear

    EYE_AR_THRESH = 0.21
    EYE_AR_CONSEC_FRAMES = 5

    # Initialize counters
    COUNTER = 0
    TOTAL = 0

    # Initialize dlib's face detector and create the facial landmark predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled/Dependencies/shape_predictor_68_face_landmarks.dat')

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    # Open the video file again for eye blink detection
    cap = cv2.VideoCapture(video_path)

    # Get the width and height of the frames in the video
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    output_path = '/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled/outputs/output - Ashish.avi'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 10, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            ear = (leftEAR + rightEAR) / 2.0

            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            if ear < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    TOTAL += 1
                COUNTER = 0

            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)

        # Write the frame to the output video file
        out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Print the final number of blinks
    print(f"Total number of blinks detected: {TOTAL}")

    if not TOTAL:
        print('UNAUTHORISED ACCESS: Spoofing detected')
    else:
        print('Moving to face detection.')

        # Initialize variables to store the highest scoring person object and its corresponding frame
        highest_score = 0
        person_bbox_with_highest_score = None
        frame_with_highest_score = None

        # Open the video file again for face detection
        cap = cv2.VideoCapture(video_path)

        # Iterate over the detected frames
        for frame_idx, frame_results in enumerate(results):
            for detection in frame_results.boxes.data:
                x1, y1, x2, y2, conf, cls = detection

                # Check if the detected object is a person (assuming class '0' corresponds to 'person')
                if int(cls) == 0 and conf > highest_score:
                    highest_score = conf
                    person_bbox_with_highest_score = (int(x1), int(y1), int(x2), int(y2))

                    # Set the video frame to the current frame index
                    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                    ret, frame_with_highest_score = cap.read()

        cap.release()

        # If a person object with the highest score is found, crop the corresponding region from the frame
        if person_bbox_with_highest_score is not None and frame_with_highest_score is not None:
            x1, y1, x2, y2 = person_bbox_with_highest_score
            cropped_person = frame_with_highest_score[y1:y2, x1:x2]
            # Display or save the cropped person image
            print(f'Frame no. {frame_idx}: {highest_score}')
            # cv2_imshow(cropped_person)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


            # --------- FACE RECOGNITION ---------

            # Path to the database directory
            # db_path = "/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/Faces/train/AshishGhule"
            db_path = "/content/drive/MyDrive/Colab Notebooks/ND Face Recognition/Faces/train/AkshayFadnavis"
            suffixes = ("png", "jpg", "jpeg")

            # Initialize variables
            is_verified = False
            threshold_value = None
            matched_image_path = None
            my_img_dist = None

            for i in os.listdir(db_path):
              if i.lower().endswith(suffixes):
                my_img_path = os.path.join(db_path, i)
                try:
                  vf = DeepFace.verify(cropped_person, my_img_path, model_name = "Facenet512", enforce_detection = False)
                  if vf['distance'] < vf['threshold']:
                    is_verified = True
                    thres = vf['threshold']
                    matched_image_path = my_img_path
                    my_img_dist = vf['distance']
                    break
                except Exception as e:
                  print(f"Error processing {my_img_path}: {e}")
                  continue

            if is_verified:
              print("ACCESS GRANTED: Face verified!")
            else:
              print("ACCESS DENIED: Face not verified!")

toc = time.time()
print(f'Total time required for code running: {round((toc-tic)/60, 2)} Minutes.')


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/91) /content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled/inputs/Ashish.mp4: 640x480 1 person, 1 bottle, 1 chair, 1 laptop, 2 keyboards, 1369.2ms
video 1/1 (frame 2/91) /content/drive/MyDrive/Colab Notebooks/ND Face Recognition/compiled/inputs/Ashish.mp4: 640x480 1 person, 1 bottle, 2 keyboards, 3309.8ms
video 1/1 (frame 3/91) /content/drive/MyDri

Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet512_weights.h5
To: /root/.deepface/weights/facenet512_weights.h5
100%|██████████| 95.0M/95.0M [00:00<00:00, 134MB/s]


ACCESS DENIED: Face not verified!
Total time required for code running: 3.42 Minutes.
